In [ ]:
import matplotlib.pyplot as plt

import torch

import os
os.chdir("/home/guest_dyw/diffusion-sampler/")

from models import get_model
from energy import get_energy_function, Plotter

from hydra import compose, initialize

MODEL_PATH = "./test/models/model copy.pt"

In [ ]:
with initialize(config_path="../configs", version_base="1.3"):
    cfg = compose(config_name="main.yaml", overrides=["model=GFN-PIS", "energy=manywell"])

In [ ]:
energy = get_energy_function(cfg)
model = get_model(cfg, energy).to('cuda:0')

model.load_state_dict(torch.load(MODEL_PATH))

In [ ]:
plotter = Plotter(
    energy, 
    plotting_bounds=(-3, 3), 
    projection_dims=[
        (i-1-((i-1) % 2), i) for i in range(1, 32)
    ],
    fig_size=(36, 100)
)

sample = model.sample(1000)

plotter.make_plot(sample)

plt.show()

In [ ]:
init_state = model.generate_initial_state(100)

traj, logpfs, logpbs = model.get_forward_trajectory(init_state)

anim, _ , _ = plotter.make_sample_generation_animation(traj, 0, 1)

In [ ]:
from IPython.display import HTML
HTML(anim.to_jshtml())